# Instituto Tecnológico y de Estudios Superiores de Monterrey
## Maestría en Inteligencia Artificial Aplicada
### Proyecto Integrador (Gpo 10) - TC5035.10

### **Proyecto: Diseño Acelerado de Fármacos**

### Avance 2: Ingeniería de características

#### **Docentes:**
- Dra. Grettel Barceló Alonso - Profesor Titular
- Dr. Luis Eduardo Falcón Morales - Profesor Titular
- Dr. Horacio Martinez Alfaro  – Profesor Tutor

#### **Miembros del equipo:**
 - Jose Luis Artigas Arroyo - A01794906
 - José Luis Fabela Portillo - A00748551
 - Alejandro Emmanuel Silva Ortega - A01794545

In [13]:
# Importar librerias
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import re
import pfeature
# from pfeature import pcp_wp
# from pfeature import ctd_wp


In [103]:
#leer csv con dataset preprocesado
df_peptidos = pd.read_csv('../data/processed/peptide_processed.csv')
df_peptidos.head(3).T

,0,1,2
Sequence,RGLRRLGRKIAHGVKKYGPTVLRIIRIA,KWKLFKKIGIGKFLHSAKKF,KWKLFKKIGPGKFLHSAKKF
Sequence_Length,28,20,20
Origin,Ovis aries,Ceropin-Magainin hybrid peptides,Ceropin-Magainin hybrid peptides
Type,Native peptide,Synthetic peptide,Synthetic peptide
Hemolytic_Activity,Human erythrocytes (4% Hemolysis=4 µM; 19% Hem...,Human erythrocytes (Hemolysis 0% =100 μM),Human erythrocytes: Hemolysis 0% (100 μM)
Cytotoxicity,NIH 3T3: IC50=16 µM,NIH-3T3 fibroblast: IC50>100 μM,NIH-3T3 fibroblast: IC50>100 μM
Binding_Target,NaN,NaN,NaN
Affinity,NaN,NaN,NaN
Mechanism,NaN,NaN,NaN
Nature,Anticancer; Antibacterial; Antifungal,Anticancer; Antibacterial,Anticancer; Antibacterial


### Eliminacion de columnas redundantes
Despues de un analis mas profundo, y una revision con el experto en el tema, se procedera a la eliminacion de columnas redundantes, ya que estos valores pueden ser obtenidos direcvtamente de la secuencia de peptidos, estas columnas son

- Chiral: Los aminoácidos L/D consisten en péptidos.
- Mass: Masa
- Formula: Formula del péptido
- Absent_amino_acids: Aminoacidos no incluidos
- Common_amino_acids: Animoacido mas comun en la sequencia
- pI: Punto isoeléctrico
- Basic_residues: Número de residuos base
- Acidic_residues: Número de residuos ácidos
- Net_charge: Carga Neta
- Polar_residues: Número de residuos polares
- Hydrophobic_residues: Número de residuos hidrofóbicos
- Hydrophobicity: Hidrofobicidad
- Boman_Index: Índice de Boman
- Half_Life: Vida media
- Aliphatic_Index: Índice alifático
- Extinction_Coefficient_cystines: Coeficiente de extinción de cistinas
- Absorbance_280nm: Absorbencia a 280nm


In [104]:
drop_cols = ['Chiral','Mass','Formula','Absent_amino_acids','Common_amino_acids','pI','Basic_residues','Acidic_residues','Net_charge','Polar_residues','Hydrophobic_residues','Hydrophobicity','Boman_Index',
             'Half_Life','Aliphatic_Index','Extinction_Coefficient_cystines','Absorbance_280nm']
df_peptidos.drop(drop_cols, axis=1, inplace=True)

In [105]:
df_peptidos.head(5).T

,0,1,2,3,4
Sequence,RGLRRLGRKIAHGVKKYGPTVLRIIRIA,KWKLFKKIGIGKFLHSAKKF,KWKLFKKIGPGKFLHSAKKF,KWKLFKKIKFLHSAKKF,KWKLFKKIPKFLHSAKKF
Sequence_Length,28,20,20,17,18
Origin,Ovis aries,Ceropin-Magainin hybrid peptides,Ceropin-Magainin hybrid peptides,Ceropin-Magainin hybrid peptides,Ceropin-Magainin hybrid peptides
Type,Native peptide,Synthetic peptide,Synthetic peptide,Synthetic peptide,Synthetic peptide
Hemolytic_Activity,Human erythrocytes (4% Hemolysis=4 µM; 19% Hem...,Human erythrocytes (Hemolysis 0% =100 μM),Human erythrocytes: Hemolysis 0% (100 μM),Human erythrocytes: Hemolysis 0% (100 μM),Human erythrocytes: Hemolysis 0% (100 μM)
Cytotoxicity,NIH 3T3: IC50=16 µM,NIH-3T3 fibroblast: IC50>100 μM,NIH-3T3 fibroblast: IC50>100 μM,NIH-3T3 fibroblast: IC50>100 μM,NIH-3T3 fibroblast: IC50>100 μM
Binding_Target,NaN,NaN,NaN,NaN,NaN
Affinity,NaN,NaN,NaN,NaN,NaN
Mechanism,NaN,NaN,NaN,NaN,NaN
Nature,Anticancer; Antibacterial; Antifungal,Anticancer; Antibacterial,Anticancer; Antibacterial,Anticancer; Antibacterial,Anticancer; Antibacterial


## Analisis de secuencias de péptidos

De acuerdo a la IUPAC[1], los aminoacidos se clasifican de acuerdo a la siguiente tabla


|symbol|Amino acid|
| :-:|:-:|
|A	| alanine|
|B	| aspartic acid or asparagine|
|C	| cysteine|
|D	| aspartic acid|
|E	| glutamic acid|
|F	| phenylalanine|
|G	| glycine|
|H	| histidine|
|I	| isoleucine|
|K	| lysine|
|L	| leucine|
|M	| methionine|
|N	| asparagine|
|P	| proline|
|Q	| glutamine|
|R	| arginine|
|S	| serine|
|T	| threonine|
|U  | selenocysteine|
|V	| valine|
|W	| tryptophan|
|X  | Desconocido u 'otro'|
|Y	| tyrosine|
|Z	| glutamic acid or glutamine|

Por lo cual, se eliminaran de la secuencia los caractares diferentes a los indicados en esta lista, de igual forma se actualizara la longitud de la secuencia para hacerla coincidir con la longitud despues de eliminar los caracteres invalidos, se eliminaran los registros con cadenas que contengan "-" y numeros (1-9) ya que estas representan peptidos modificados y contienen compuestos quimicos los cuales modifican el péptido y estos no seran considerados para el modelo, de igual forma se eliminaran las secuencias que contengan el caracter Ⓧ ya que este caracter representa peptidos ciclicos los cuales no se tomaran en cuenta para este modelo ya que no representan una muestra significativa.

In [106]:
df_peptidos["Sequence"]

0                RGLRRLGRKIAHGVKKYGPTVLRIIRIA
1                        KWKLFKKIGIGKFLHSAKKF
2                        KWKLFKKIGPGKFLHSAKKF
3                           KWKLFKKIKFLHSAKKF
4                          KWKLFKKIPKFLHSAKKF
                        ...                  
5846                 FLPLLAGLAANFLPKIICKIARKC
5847                  RIIDRLWLVRRPOKPKFVLVWVL
5848                               FHAVPQSFYT
5849                            KLWCKSSQVPQSR
5850    TKEQKEQIAKATGLTTKQVRNWYVQLNASIKVCMCSC
Name: Sequence, Length: 5851, dtype: object

In [107]:
df_peptidos["Sequence"].map(lambda x: re.sub(r"[\"]",'',x))

0                RGLRRLGRKIAHGVKKYGPTVLRIIRIA
1                        KWKLFKKIGIGKFLHSAKKF
2                        KWKLFKKIGPGKFLHSAKKF
3                           KWKLFKKIKFLHSAKKF
4                          KWKLFKKIPKFLHSAKKF
                        ...                  
5846                 FLPLLAGLAANFLPKIICKIARKC
5847                  RIIDRLWLVRRPOKPKFVLVWVL
5848                               FHAVPQSFYT
5849                            KLWCKSSQVPQSR
5850    TKEQKEQIAKATGLTTKQVRNWYVQLNASIKVCMCSC
Name: Sequence, Length: 5851, dtype: object

In [108]:
# Crear un diccionario para contar la aparición de cada carácter de la secuencia de aminoácidos
char_count = {}

# Iterar sobre cada secuencia en la columna 'Sequence'
for sequence in df_peptidos['Sequence']:
    # Convertir la secuencia en un conjunto de caracteres únicos
    unique_chars = set(sequence)
    # Contar la aparición de cada carácter en el conjunto
    for char in unique_chars:
        if char in char_count:
            char_count[char] += 1
        else:
            char_count[char] = 1

# Convertir el diccionario en un DataFrame para una mejor visualización
char_count_df = pd.DataFrame(list(char_count.items()), columns=['Character', 'Count'])
char_count_df['%'] = (char_count_df['Count'] / df_peptidos.shape[0]).round(4) *100

# Número de péptidos por caracter
char_count_df.T

,0,1,2,3,4,5,6,7,8,9,...,54,55,56,57,58,59,60,61,62,63
Character,Y,A,I,P,T,L,K,G,V,R,...,Φ,(,),[,],b,1,2,6,7
Count,1985,3014,3081,2435,1684,3858,3390,2964,2932,3207,...,11,4,4,1,1,1,1,1,1,1
%,33.93,51.51,52.66,41.62,28.78,65.94,57.94,50.66,50.11,54.81,...,0.19,0.07,0.07,0.02,0.02,0.02,0.02,0.02,0.02,0.02


Es importante identificar los caracteres contenidos en la secuencia de aminoácidos de los péptidos, por tal motivo vamos a analizar su ocurrencia en el total de péptidos. La lista utilizada es la que se describio anteriormente (IUPAC), sin embargo, para obtener las características con la librería de Pfeatures no se reconocen 4 de estos: B, U, X, Z. Asímismo existen otros caracteres no incluidos en IUPAC por lo que es necesario analizar que tan frecuentes son.

In [109]:
# Aminoácidos Pfeature
aa_pfeature = ['A','C','D','E','F','G','H','I','K','L','M','N','P',
              'Q','R','S','T','V','W','Y']

# Aminoácidos IUPAC - Pfeatures
aa_iupac = ['B','U','X','Z']

# Aminoácidos en minúsculas
aa_min = [elemento.lower() for elemento in (aa_pfeature + aa_iupac)]

print ('Total de aminoácidos IUPAC:', len(aa_iupac) + len(aa_pfeature))
print ('Total de aminoácidos Pfeatures:', len(aa_pfeature))
print ('Total de aminoácidos que están en UIPAC pero no reconoce Pfeatures:', len(aa_iupac))



Total de aminoácidos IUPAC: 24
Total de aminoácidos Pfeatures: 20
Total de aminoácidos que están en UIPAC pero no reconoce Pfeatures: 4


In [110]:
# Crear un diccionario para contar la aparición de cada carácter
char_count = {}

# Iterar sobre cada secuencia en la columna 'Sequence'
for sequence in df_peptidos['Sequence']:
    # Convertir la secuencia en un conjunto de caracteres únicos
    unique_chars = set(sequence)
    # Contar la aparición de cada carácter en el conjunto
    for char in unique_chars:
        if char in char_count:
            char_count[char] += 1
        else:
            char_count[char] = 1

# Convertir el diccionario en un DataFrame para una mejor visualización
char_count_df = pd.DataFrame(list(char_count.items()), columns=['Character', 'Count'])
char_count_df['%'] = (char_count_df['Count'] / df_peptidos.shape[0]).round(4) *100


print('Total de caracteres únicos: ', len(char_count_df))

Total de caracteres únicos:  64


In [111]:
# Aminoácidos Pfeatures Mayúsculas
char_count_aapf = char_count_df[char_count_df['Character'].isin(aa_pfeature)]
print('Total de caracteres de Pfeatures incluidos en el dataset: ', len(char_count_aapf))
char_count_aapf.sort_values(by='Count', ascending=False).T

Total de caracteres de Pfeatures incluidos en el dataset:  20


,5,6,9,2,1,7,8,11,3,12,13,0,16,14,15,17,4,19,18,10
Character,L,K,R,I,A,G,V,F,P,W,S,Y,N,D,E,Q,T,M,C,H
Count,3858,3390,3207,3081,3014,2964,2932,2875,2435,2143,2077,1985,1942,1825,1824,1808,1684,1202,1116,1033
%,65.94,57.94,54.81,52.66,51.51,50.66,50.11,49.14,41.62,36.63,35.5,33.93,33.19,31.19,31.17,30.9,28.78,20.54,19.07,17.66


In [112]:
# Aminoácidos UIPAC no incluídos en Pfeatures Mayúsculas
char_count_aauipac = char_count_df[char_count_df['Character'].isin(aa_iupac)]
print('Total de caracteres de Pfeatures incluidos en el dataset: ', len(char_count_aauipac))
char_count_aauipac.sort_values(by='Count', ascending=False).T

Total de caracteres de Pfeatures incluidos en el dataset:  4


,29,51,37,41
Character,X,Z,B,U
Count,691,364,31,5
%,11.81,6.22,0.53,0.09


La ocurrencia de Z, B, U es muy baja por lo que se procederá a eliminar todos los péptidos que las contienen. 
Para el caso de los péptidos con 'x' se re-escribiran aquellas que solo tengan X al inicio o al final. Por ejemplo, si la secuencia es XXVKKGLISY reescribirla como VKKGLISY, si la secuencia es KKGLISYXXX entonces se reescribirá como KKGLISY. Sin embargo, si la secuencia es KKGXXLISY entonces se eliminará la secuencia del dataset.

In [113]:
# Aminoácidos UIPAC no incluídos en Pfeatures Mayúsculas
char_count_aamin = char_count_df[char_count_df['Character'].isin(aa_min)]
print('Total de caracteres de Pfeatures incluidos en el dataset: ', len(char_count_aamin))
char_count_aamin.sort_values(by='Count', ascending=False).T

Total de caracteres de Pfeatures incluidos en el dataset:  24


,30,25,24,28,39,22,23,32,21,34,...,36,26,47,43,33,49,20,44,53,59
Character,r,l,k,f,x,a,v,w,i,n,...,t,g,q,e,m,h,d,u,z,b
Count,141,132,112,78,77,54,54,52,44,33,...,21,19,15,13,11,8,7,1,1,1
%,2.41,2.26,1.91,1.33,1.32,0.92,0.92,0.89,0.75,0.56,...,0.36,0.32,0.26,0.22,0.19,0.14,0.12,0.02,0.02,0.02


La distincion entre los tipos de aminoácidos L o sinteticos y D o naturales se puede ver en estos resultados, siendo representados los tipo D con letras mayusculuas y los tipo L con minusculas. Gracias a este analisis podemos mostrar que no es relevante mantener la distincion entre estos aminoácidos y se hara un tratamiento a estos datos para convertir los caracteres en minusculas a mayusculas mas adelante.  

In [114]:
# Número de péptidos en los que está cada caracter que no es aminoácido IUPAC:
char_count_otros = char_count_df[~char_count_df['Character'].isin(aa_pfeature + aa_iupac + aa_min)]
print('Total de caracteres no incluidos en el UIPAC: ', len(char_count_otros))
char_count_otros.sort_values(by='Count', ascending=False).T

Total de caracteres no incluidos en el UIPAC:  16


,46,52,54,31,55,56,45,50,42,38,57,58,60,61,62,63
Character,Ⓧ,",",Φ,O,(,),-,j,,工,[,],1,2,6,7
Count,236,18,11,10,4,4,3,3,2,1,1,1,1,1,1,1
%,4.03,0.31,0.19,0.17,0.07,0.07,0.05,0.05,0.03,0.02,0.02,0.02,0.02,0.02,0.02,0.02


In [115]:
char_count_otros['Character'].unique()

array(['O', '工', '\xa0', '-', 'Ⓧ', 'j', ',', 'Φ', '(', ')', '[', ']', '1',
       '2', '6', '7'], dtype=object)

La frecuencia de los caractéres no incluidos en IUPAC es muy baja, por lo que también se eliminarán los péptidos que las contienen.

In [133]:
# Ajuste de las secuencias de aminoácidos
filtered_df = df_peptidos.copy()
# 1. Poner en mayúsculas todos los aminoácidos de las secuencias
filtered_df['Sequence'] = filtered_df['Sequence'].str.upper()
# 2. Identificar los péptidos con valores de X
aa_x = filtered_df[filtered_df['Sequence'].apply(lambda x: 'X' in x)]
# 3. Conservar solo los péptidos que contengan los aminoácidos para pfeatures
filtered_df = filtered_df[filtered_df['Sequence'].apply(lambda x: all(char in aa_pfeature for char in x))]

In [134]:
# Identifica todos péptidos con valores de X
total_x = len(aa_x)
print('Total de péptidos con X', total_x)

# Inician con X 
inicio_x = aa_x[aa_x['Sequence'].str.startswith('X')]
print('Péptidos que empiezan con X: ', len(inicio_x))
# Función para quitar los caracteres 'x' al inicio de la secuencia
def quitar_x_inicio(secuencia):
    return re.sub(r'^X+', '', secuencia)
# Aplicar la función a la columna 'Sequence'
aa_x['Sequence'] = aa_x['Sequence'].apply(quitar_x_inicio)

# Finalizan con X
fin_x = aa_x[aa_x['Sequence'].str.endswith('X')].copy()
print('Péptidos que finalizan con X: ', len(fin_x))
# Función para quitar los caracteres 'x' al final de la secuencia
def quitar_x_final(secuencia):
    return re.sub(r'X+$', '', secuencia)
# Aplicar la función a la columna 'Sequence'
aa_x['Sequence'] = aa_x['Sequence'].apply(quitar_x_final)

# Elimina los péptidos con X intermedia
aa_x = aa_x[aa_x['Sequence'].apply(lambda x: all(char in aa_pfeature for char in x))]
print('Péptidos con X intermedia:', total_x - len(aa_x))
print('Peptidos conservados: ', len(aa_x))

Total de péptidos con X 727
Péptidos que empiezan con X:  257
Péptidos que finalizan con X:  177
Péptidos con X intermedia: 551
Peptidos conservados:  176


In [135]:
# Agrega los peptidos con X modificados al dataframe de péptidos depurados
filtered_df = pd.concat([filtered_df, aa_x], ignore_index=True)

print('Total de péptidos: ', df_peptidos.shape[0])
print('Péptidos solo con aminoácidos para pFeatures: ', filtered_df.shape[0])
print('Péptidos eliminados: ', round((df_peptidos.shape[0]-filtered_df.shape[0])/df_peptidos.shape[0],4) * 100, '%')

Total de péptidos:  5851
Péptidos solo con aminoácidos para pFeatures:  4679
Péptidos eliminados:  20.03 %


In [136]:
# Valida los aminoácidos contenidos
char_count = {}

# Iterar sobre cada secuencia en la columna 'Sequence'
for sequence in filtered_df['Sequence']:
    # Convertir la secuencia en un conjunto de caracteres únicos
    unique_chars = set(sequence)
    # Contar la aparición de cada carácter en el conjunto
    for char in unique_chars:
        if char in char_count:
            char_count[char] += 1
        else:
            char_count[char] = 1

# Convertir el diccionario en un DataFrame para una mejor visualización
char_count_df = pd.DataFrame(list(char_count.items()), columns=['Character', 'Count'])
char_count_df['%'] = (char_count_df['Count'] / filtered_df.shape[0]).round(4) *100


print('Total de caracteres únicos: ', len(char_count_df))

Total de caracteres únicos:  20


In [137]:
filtered_df.to_csv('../data/processed/peptide_filtered.csv', index = False)

### Convertir dataframe to FASTA 

Para los siguientes pasos del proceso, es necesario hacer la conversion de los datos filtrados a formato FASTA, este formato es el estandard usado para las secuencias de aminoácidos

In [138]:

with open('../data/processed/VEGF.fasta','w', encoding='UTF8') as f:
    for _, row in filtered_df.iterrows():
        f.write(f">a{row['VEGF']}\n{row['Sequence']}\n")
    


### Ejecutar CD-HIT

In [139]:
#Ejecutar cd-hit en colab, para windows usar el comando con wsl
#! cd-hit -i data/processed/VEGF.fasta -o data/processed/VEGF.txt -c 0.99

import subprocess

# Define the WSL command buscar 
command = ["wsl", "cd-hit", "-i", "../data/processed/VEGF.fasta", "-o", "../data/processed/VEGF.txt", "-c", "0.99"]

# Run the command
result = subprocess.run(command, capture_output=True, text=True)

# Print the output and errors
print("STDOUT:", result.stdout)
print("STDERR:", result.stderr)


STDOUT: 
 
 
 C o p y r i g h t   ( c )   M i c r o s o f t   C o r p o r a t i o n .   T o d o s   l o s   d e r e c h o s   r e s e r v a d o s . 
 
 
 
 
 
   U s o :   w s l . e x e   [ A r g u m e n t ] 
 
 
 
 
 
 A r g u m e n t o s : 
 
 
 
 
 
         - - i n s t a l l   < O p t i o n s > 
 
 
                 I n s t a l a   e l   S u b s i s t e m a   d e   W i n d o w s   p a r a   l a s   c a r a c t e r í s t i c a s   d e   L i n u x .   S i   n o   s e   e s p e c i f i c a n   o p c i o n e s , 
 
 
                 l a s   c a r a c t e r í s t i c a s   r e c o m e n d a d a s   s e   i n s t a l a r á n   j u n t o   c o n   l a   d i s t r i b u c i ó n   p r e d e t e r m i n a d a . 
 
 
 
 
 
             P a r a   v e r   l a   d i s t r i b u c i ó n   p r e d e t e r m i n a d a ,   a s í   c o m o   u n a   l i s t a   d e   o t r a s   d i s t r i b u c i o n e s   v á l i d a s , 
 
 
 u s e   ' w s l   - - l i s t   - - o n l i n e ' . 
 
 
 
 
 
       

# Funciones de Pfeature

In [140]:
cd_hit_path = "../data/processed/VEGF.txt"

### Physico-Chemical Properties Composition: 30 Descriptores

Este modulo calcula la fracción de cada propiedas físico-química de una secuencia. Las siguientes propiedades se han incorporado a Pfeature para calcular caracteristicas composicionales basadas en propiedades físico-químicas estandard.

- Positively Charged → Carga Positiva
- Negatively Charged → Carga Negativa
- Neutral Charge → Carga Neutra
- Polarity in residues → Polaridad en residuos
- Non-polarity in residues → No polaridad en residuos
- Aliphaticity → Alifaticidad
- Cyclicity → Ciclicidad
- Aromaticity → Aromaticidad
- Acidity → Acidez
- Basicity → Basicidad
- Neutral (pH) → Neutro (pH)
- Hydrophobicity → Hidrofobicidad
- Hydrophilicity → Hidrofilicidad
- Neutral towards water → Neutro frente al agua
- Hydroxylic → Hidroxílico
- Sulphur Content → Contenido de Azufre
- Tiny → Pequeño
- Small → Mediano
- Large → Grande

parfa el cálculo de estas propiedades se usa la siguiente formula 𝐶𝑃𝑖 = 𝑃𝑖/𝐿 donde
𝐶𝑃𝑖 = Propiedades físico-quimicas de la composicion de tipo 𝑖.
𝑃𝑖 = suma de la propiedad de tipo 𝑖.
𝐿 = Longitud de la secuencia. 

In [141]:
# Physico-Chemical Properties Composition (PCP)

def pcp(input):
  a = input.rstrip('txt')
  output = a + 'dpc.csv'
  df_out = pcp_wp(input, output)
  df_in = pd.read_csv(output)
  return df_in

feature_pcp = pcp(cd_hit_path)
feature_pcp

NameError: name 'pcp_wp' is not defined

### Composition enhanced Transition and Distribution: 189 descriptors [2]

Composition-Enhanced Transition and Distribution (CTD) es un enfoque utilizado en bioinformática y química computacional para describir secuencias de aminoácidos o nucleótidos. Se basa en tres principios clave:

Composición (Composition): Representa la proporción de cada tipo de aminoácido o nucleótido en una secuencia.
Transición (Transition): Describe la frecuencia con la que un tipo de aminoácido/nucleótido cambia a otro a lo largo de la secuencia.
Distribución (Distribution): Indica la posición relativa de ciertos tipos de aminoácidos/nucleótidos dentro de la secuencia.

Cómo funciona CTD en la descripción de secuencias

1. Composición (C - Group 1) 

- Calcula el porcentaje de cada grupo de aminoácidos en la secuencia.
- Se agrupan los aminoácidos según ciertas propiedades fisicoquímicas, como hidrofobicidad o carga.

2. Transición (T - Group 2)

- Representa la frecuencia de cambio entre dos grupos de aminoácidos consecutivos en la secuencia.
- Se calcula como el número de transiciones entre los grupos dividido por el total de transiciones posibles.

3. Distribución (D - Group 3)

- Indica en qué posiciones de la secuencia aparecen los primeros, el 25%, 50%, 75% y 100% de los residuos de un grupo específico.

|Attributes|Group1|Group 2|Group 3|
|:-:|:-:|:-:|:-:|
|Hydrophobicity| R,K,E,D,Q,N| G,A,S,T,P,H,Y| C,L,V,I,M,F,W|
|Normalized Vander Waals volume| G,A,S,T,P,D| N,V,E,Q,I,L| M,H,K,F,R,Y,W|
|Polarity| L,I,F,W,C,M,V,Y| P,A,T,G,S| H,Q,R,K,N,E,D|
|Polarizability| G,A,S,D,T| C,P,N,V,E,Q,I,L| K,M,H,F,R,Y,W|
|Charge| K,R| A,N,C,Q,G,H,I,L,M,F,P,S,T,W,Y,V| D,E|
|Secondary structure| E,A,L,M,Q,K,R,H| V,I,Y,C,W,F,T| G,N,P,S,D|
|Solvent accessibility| A,L,F,C,G,I,V,W| R,K,Q,E,N,D| M,S P,T,H,Y|

Los atributos de CTD permiten capturar información estructural y funcional de secuencias biológicas, convirtiéndolas en vectores numéricos que pueden ser utilizados para la predicción y clasificación en bioinformática.

In [17]:
# Composition enhanced Transition and Distribution: (CTD) 

def ctd(input):
  a = input.rstrip('txt')
  output = a + 'ctd.csv'
  df_out = ctd_wp(input, output)
  df_in = pd.read_csv(output)
  return df_in

feature_ctd = ctd(cd_hit_path)
feature_ctd

,CeTD_HB1,CeTD_HB2,CeTD_HB3,CeTD_VW1,CeTD_VW2,CeTD_VW3,CeTD_PO1,CeTD_PO2,CeTD_PO3,CeTD_PZ1,...,CeTD_0_p_SS3,CeTD_25_p_SS3,CeTD_50_p_SS3,CeTD_75_p_SS3,CeTD_100_p_SS3,CeTD_0_p_SA3,CeTD_25_p_SA3,CeTD_50_p_SA3,CeTD_75_p_SA3,CeTD_100_p_SA3
0,32.14,35.71,32.14,28.57,32.14,39.29,35.71,28.57,35.71,25.00,...,0,2,4,6,9,0,1,2,3,4
1,35.00,25.00,40.00,20.00,20.00,60.00,40.00,20.00,40.00,20.00,...,0,1,3,5,7,0,0,1,1,2
2,35.00,30.00,35.00,25.00,15.00,60.00,35.00,25.00,40.00,20.00,...,0,1,3,5,7,0,0,1,2,3
3,38.89,22.22,38.89,16.67,16.67,66.67,38.89,16.67,44.44,11.11,...,0,1,3,5,7,0,0,1,2,3
4,18.75,31.25,50.00,37.50,43.75,18.75,50.00,31.25,18.75,37.50,...,0,0,1,2,3,0,0,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2651,50.00,20.00,30.00,20.00,35.00,45.00,30.00,20.00,50.00,15.00,...,0,2,5,7,10,0,0,1,1,2
2652,20.83,29.17,50.00,31.82,45.45,22.73,50.00,29.17,20.83,20.83,...,0,1,2,3,5,0,0,1,1,2
2653,20.83,29.17,50.00,31.82,40.91,27.27,50.00,29.17,20.83,20.83,...,0,1,2,3,5,0,0,1,1,2
2654,38.46,30.77,30.77,33.33,33.33,33.33,30.77,30.77,38.46,23.08,...,0,1,2,3,5,0,1,2,3,4


### Referencias

1. INTERNATIONAL UNION OF PURE AND APPLIED CHEMISTRY (IUPAC). https://iupac.qmul.ac.uk/AminoAcid/A2021.html#AA20
2. Dubchak I, et al. Proceedings of the National Academy of Sciences of the United States of America